In [49]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import cmocean as cmo
from matplotlib.patches import Polygon
from copy import copy
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

In [50]:
lw_voronoi = 0.
lw_gl = .2
ec_voronoi = None #'w'

plotannual = True

In [51]:
#    C%type_icefree_land                        = 1
#    C%type_icefree_ocean                       = 2
#    C%type_grounded_ice                        = 3
#    C%type_floating_ice                        = 4
#    C%type_groundingline_gr                    = 5
#    C%type_groundingline_fl                    = 6
#    C%type_calvingfront_gr                     = 7
#    C%type_calvingfront_fl                     = 8
#    C%type_margin                              = 9
#    C%type_coastline                           = 10

In [ ]:
vmax = 100
vmin = -3
linthresh = 1
linscale = .2

fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
nneg = np.int_((1-fracpos)*256)

hilim = 100
N = np.int_(np.log10(vmax/hilim)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale) * 256)
print(N)

colors1 = plt.get_cmap('cmo.dense_r')(np.linspace(1-nneg/(256-nneg),1.,nneg+1))
colors2 = plt.get_cmap('gist_heat_r')(np.linspace(.02, 1, 256-nneg-1))
colors3 = plt.get_cmap('PuRd_r')(np.linspace(0., .5, N))

#colors1 = plt.get_cmap('cmo.ice_r')(np.linspace(0,1.,nneg+1))
#colors2 = plt.get_cmap('inferno')(np.linspace(0., 1, 256-nneg-1))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2, colors3))

cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)
norm1=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale)

ticks1 = [-3,-1,0,1,3,10,30,100]

In [ ]:
run = 'Antarctica'
s = 1

fig = plt.figure(figsize=(7,10))

ax0 = fig.add_subplot(111)

#Add colorbars
cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm1, cmap=cmap1),ax=ax0,extend='both',shrink=.8)
cb.set_ticks(ticks1)
cb.set_ticklabels(ticks1)
cb.set_label("BMB [m.i.e./yr]")

ax0.set_xticklabels([])
ax0.set_yticklabels([])

#Open data on new mesh
ds = xr.open_dataset(f'../output/{run}/main_output_ANT_{s:05d}.nc')

#Make up axes and labels
ax0.set_xlim(ds.xmin+2e3,ds.xmax-2e3)
ax0.set_ylim(ds.ymin+2e3,ds.ymax-2e3)
ax0.set_aspect(1)
ax0.grid(color='.3',linewidth=.1)

#Allocate for storage of polygons
BMB = {}

# Gather points surrounding vertex (quasi-voronoi)
for v in range(len(ds.vi)):
    polyx = []
    polyy = []
    for n in range(ds.niTri[v].data):
        c = ds.iTri[n,v].data-1 #Neighbouring triangle
        polyx.append(ds.Tricc[0,c].data)
        polyy.append(ds.Tricc[1,c].data)             
    #TO DO: Append edge points here

    #Accumulate into polygons
    points = np.array([polyx,polyy]).T #Converted to km
    BMB[v] = Polygon(points,edgecolor=ec_voronoi,linewidth=lw_voronoi)

    #Add polygons to axes
    ax0.add_patch(BMB[v])

print(f"{s:02d}: Added cells")

i = 0
#Loop over vertex to fill colours and plot grounding line
for v in range(len(ds.vi)):

    #Fill colours based on data
    if ds.mask[i,v].data in [2]: #Ocean
        BMB[v].set_facecolor('teal')
    elif ds.mask[i,v].data in [4,6,8]: #Ice shelf
        BMB[v].set_facecolor(cmap1(norm1(-ds.BMB[i,v])))
    elif (ds.mask[i,v].data in [5]): # and ds.BMB[i,v] != 0): #gl_gr
        BMB[v].set_facecolor('.8')#cmap1(norm1(-ds.BMB[i,v]))) #Grounded with possible BMB from PMP
    else: #Grouded
        BMB[v].set_facecolor('.8')
    
    # Plot grounding line segments
    if ds.mask[i,v].data in [5,7]: #gl_gr or ca_gr
        for n in range(ds.nC[v].data):
            c = ds.C[n,v].data-1
            if ds.mask[i,c].data in [6,8]: #neighbouring gl_fl or ca_fl
                cs = np.intersect1d(ds.iTri[:,v],ds.iTri[:,c])[1:]-1
                ax0.plot(ds.Tricc[0,cs]/1000,ds.Tricc[1,cs]/1000,lw=lw_gl,c='yellow')
            elif ds.mask[i,c].data in [2]: #Ocean
                cs = np.intersect1d(ds.iTri[:,v],ds.iTri[:,c])[1:]-1
                ax0.plot(ds.Tricc[0,cs]/1000,ds.Tricc[1,cs]/1000,lw=lw_gl,c='orange')                       

#Save figure
plt.savefig(f'../video/fig_{run}.png',dpi=1200)

ds.close()
